In [1]:
!pip install open3d scipy spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import open3d as o3d
import tensorflow as tf
import numpy as np
import spektral
from tensorflow.keras.layers import Dropout, Input
from scipy.sparse import csr_matrix
path ="/content/patch_523.ply"

In [3]:
pcd = o3d.io.read_point_cloud(path)
pcd

PointCloud with 2048 points.

In [4]:
pcd_xyz = np.asarray(pcd.points)
pcd_rgb = np.asarray(pcd.colors)
full_pc = np.concatenate((pcd_xyz,pcd_rgb),axis=1)

In [5]:
A = csr_matrix(full_pc)
A

<2048x6 sparse matrix of type '<class 'numpy.float64'>'
	with 12288 stored elements in Compressed Sparse Row format>

In [6]:
polynominal = spektral.utils.chebyshev_polynomial(csr_matrix(full_pc), 1 )

In [7]:
#          Parameters 
conv_layers =[512 , 256 , 128 , 64 , 1]
qps = [51 , 46 , 40 , 34]
# Learning Rate
lr=1e-5
# Batch
batch_size = 8
# Adam
beta1 = 0.9
beta2=0.999
# Inputs
F = 6 # Features
N= 2048 # Patch

In [8]:
# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True, dtype=object)

In [9]:
from spektral.layers import ChebConv
from spektral.layers import GATConv
from tensorflow.keras.regularizers import l2

gc_1 = ChebConv(
    64, K=1, activation="relu", kernel_regularizer=l2(2.5e-4), use_bias=False
)([x_in, a_in])

gat_1 = GATConv(
    64,
    concat_heads=True,
    activation="elu",
    kernel_regularizer=l2(2.5e-4),
    attn_kernel_regularizer=l2(2.5e-4),
    bias_regularizer=l2(2.5e-4),
)([gc_1, a_in])

gc_2 = ChebConv(
    64, K=1, activation="relu", kernel_regularizer=l2(2.5e-4), use_bias=False
)([gat_1, a_in])

gat_2 = GATConv(
    64,
    concat_heads=True,
    activation="elu",
    kernel_regularizer=l2(2.5e-4),
    attn_kernel_regularizer=l2(2.5e-4),
    bias_regularizer=l2(2.5e-4),
)([gc_2, a_in])

gc_3 = ChebConv(
    64, K=1, activation="relu", kernel_regularizer=l2(2.5e-4), use_bias=False
)([gat_2, a_in])

conc = tf.keras.layers.Concatenate(axis=1)([gc_1 , gat_1,gc_2 , gat_2,gc_3])
dense = tf.keras.layers.Dense(64)(conc)

In [10]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
# Build model
branch = Model(inputs=[x_in, a_in], outputs=dense)
optimizer = Adam(lr=lr)
branch.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(reduction="sum"),  # To compute mean
    weighted_metrics=["acc"],
)
branch.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 cheb_conv (ChebConv)           (None, 64)           384         ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 gat_conv (GATConv)             (None, 64)           4288        ['cheb_conv[0][0]',          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
conc

<KerasTensor: shape=(None, 320) dtype=float32 (created by layer 'concatenate')>